In [ ]:
import cv2
import numpy as np
import xarray as xr

# Función para imprimir las propiedades del objeto
def imprimir_propiedades_objeto(objeto, imagen_hsv):
    print("id", objeto["id"])
    print("Área:", objeto["area"])
    print("Perímetro:", objeto["perimetro"])
    print("Coordenadas:", (objeto["contorno_x"], objeto["contorno_y"], objeto["ancho"], objeto["alto"]))
    print("Centroide:", (objeto["centroide_x"], objeto["centroide_y"]))
    print("Ángulo:", objeto["orientacion"])
    print("Latitud:", objeto["latitud"])
    print("Longitud:", objeto["longitud"])
    print("Intensidad completa:", imagen_hsv[objeto["contorno_y"]:objeto["contorno_y"]+objeto["alto"], objeto["contorno_x"]:objeto["contorno_x"]+objeto["ancho"], 2])
    print("\n")

# Función para ajustar el brillo y contraste de la imagen
def aumentar_color(imagen, brillo=0, contraste=1.0):
    # Código para ajustar el brillo y contraste de la imagen aquí
    imagen_aumentada = cv2.convertScaleAbs(imagen, alpha=contraste, beta=brillo)
    return imagen_aumentada

# Ruta del archivo NetCDF con los datos de intensidad de precipitación
ruta_netcdf = '/home/sebastian/Documentos/analisisNETCDF/ivt_s_6_2022_Jan.nc'

# Cargar los datos de intensidad de precipitación desde el archivo NetCDF
precip_da = xr.open_dataarray(ruta_netcdf)

# Ruta de la imagen con los objetos detectados
ruta_imagen = '/home/sebastian/Documentos/analisisNETCDF/datosGraficados/2022-01-01.png'

# Cargar la imagen
imagen_simulada = cv2.imread(ruta_imagen)

# Convertir la imagen al espacio de color HSV
imagen_hsv = cv2.cvtColor(imagen_simulada, cv2.COLOR_BGR2HSV)

# Definir el rango de valores de color azul en HSV
rango_bajo = np.array([90, 50, 50])
rango_alto = np.array([130, 255, 255])

# Crear una máscara binaria de los píxeles azules
mascara = cv2.inRange(imagen_hsv, rango_bajo, rango_alto)

# Aplicar operaciones morfológicas para eliminar el ruido
kernel = np.ones((5, 5), np.uint8)
mascara = cv2.morphologyEx(mascara, cv2.MORPH_OPEN, kernel)
mascara = cv2.morphologyEx(mascara, cv2.MORPH_CLOSE, kernel)

# Encontrar los contornos en la imagen binarizada
contornos, _ = cv2.findContours(mascara, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Crear una lista para almacenar las propiedades de los objetos
propiedades_objetos = []
contador = 0

# Recorrer todos los objetos encontrados
for i, contorno in enumerate(contornos):
    # Calcular el área del objeto
    area = cv2.contourArea(contorno)

    # Calcular el perímetro del objeto
    perimetro = cv2.arcLength(contorno, True)

    # Calcular las coordenadas del rectángulo que encierra al objeto
    (x, y, w, h) = cv2.boundingRect(contorno)

    # Calcular las coordenadas del centroide del objeto
    momentos = cv2.moments(contorno)
    cX = int(momentos["m10"] / momentos["m00"])
    cY = int(momentos["m01"] / momentos["m00"])

    # Verificar si el contorno tiene suficientes puntos para ajustar una elipse
    if len(contorno) >= 5:
        # Calcular el ángulo de orientación del objeto
        (pos_x, pos_y), (maj_axis, min_axis), angle = cv2.fitEllipse(contorno)
    else:
        angle = 0.0

    # Calcular las coordenadas de latitud y longitud
    latitud = 20 - (i * 10)
    longitud = -90 - (x * 10)

    # Calcular el ancho del objeto
    ancho = w

    # Calcular la relación de ancho del objeto
    relacion_ancho = w / h

    # Calcular la intensidad promedio del objeto
    intensidad = np.mean(imagen_hsv[y:y+h, x:x+w, 2])

    # Incrementar el contador de objetos
    contador += 1

    # Dibujar el número del objeto sobre el contorno
    cv2.putText(imagen_simulada, str(contador), (x + 10, y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # Dibujar el contorno del objeto en la imagen
    cv2.drawContours(imagen_simulada, [contorno], -1, (0, 255, 0), 2)

    # Crear un diccionario con las propiedades del objeto
    objeto = {
        "id": contador,
        "area": area,
        "perimetro": perimetro,
        "contorno_x": x,
        "contorno_y": y,
        "ancho": w,
        "alto": h,
        "centroide_x": cX,
        "centroide_y": cY,
        "orientacion": angle,
        "latitud": latitud,
        "longitud": longitud,
        "relacion_ancho": relacion_ancho,
        "intensidad": intensidad
    }

    # Agregar el diccionario a la lista de propiedades de objetos
    propiedades_objetos.append(objeto)

    # Imprimir las propiedades del objeto actual
    imprimir_propiedades_objeto(objeto, imagen_hsv)

# Mostrar la imagen con los objetos detectados y sus contornos numerados
cv2.imshow("Objetos Detectados con Contornos Numerados", imagen_simulada)
cv2.waitKey(0)
cv2.destroyAllWindows()


id 1
Área: 153.0
Perímetro: 55.79898929595947
Coordenadas: (695, 516, 20, 14)
Centroide: (705, 522)
Ángulo: 67.66337585449219
Latitud: 20
Longitud: -7040
Intensidad completa: [[255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255 255
  255 255]
 [255 255 255 255 255 255 255 255 255 255 255 255 255 255 25

QObject::moveToThread: Current thread (0x2a9f700) is not the object's thread (0x2e271c0).
Cannot move to target thread (0x2a9f700)

QObject::moveToThread: Current thread (0x2a9f700) is not the object's thread (0x2e271c0).
Cannot move to target thread (0x2a9f700)

QObject::moveToThread: Current thread (0x2a9f700) is not the object's thread (0x2e271c0).
Cannot move to target thread (0x2a9f700)

QObject::moveToThread: Current thread (0x2a9f700) is not the object's thread (0x2e271c0).
Cannot move to target thread (0x2a9f700)

QObject::moveToThread: Current thread (0x2a9f700) is not the object's thread (0x2e271c0).
Cannot move to target thread (0x2a9f700)

QObject::moveToThread: Current thread (0x2a9f700) is not the object's thread (0x2e271c0).
Cannot move to target thread (0x2a9f700)

QObject::moveToThread: Current thread (0x2a9f700) is not the object's thread (0x2e271c0).
Cannot move to target thread (0x2a9f700)

QObject::moveToThread: Current thread (0x2a9f700) is not the object's thread